# AI Agent Demo: Automated Forecasting Workflow Generation

This notebook demonstrates the `py_agent` package for automated time series forecasting.

The `ForecastAgent` can:
- Analyze your data automatically
- Recommend appropriate models
- Generate preprocessing recipes
- Create complete workflows
- Debug performance issues

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

from py_agent import ForecastAgent

print("✅ Imports successful")

## Example 1: Simple Daily Sales Forecasting

Let's create synthetic daily sales data with weekly seasonality.

In [ ]:
# Create synthetic sales data
np.random.seed(42)
dates = pd.date_range('2020-01-01', periods=365*2, freq='D')

# Components: trend + weekly seasonality + noise
trend = np.arange(len(dates)) * 0.1
seasonality = np.sin(np.arange(len(dates)) * 2 * np.pi / 7) * 20
noise = np.random.randn(len(dates)) * 5
sales = 100 + trend + seasonality + noise

# Create DataFrame
df = pd.DataFrame({
    'date': dates,
    'sales': sales
})

# Split train/test
train = df.iloc[:int(len(df)*0.8)]
test = df.iloc[int(len(df)*0.8):]

print(f"Train: {len(train)} days")
print(f"Test: {len(test)} days")
print(f"\nFirst few rows:")
print(train.head())

In [ ]:
# Visualize the data
plt.figure(figsize=(12, 4))
plt.plot(train['date'], train['sales'], label='Train', alpha=0.7)
plt.plot(test['date'], test['sales'], label='Test', alpha=0.7)
plt.xlabel('Date')
plt.ylabel('Sales')
plt.title('Daily Sales Data')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### Generate Workflow with AI Agent

Simply describe what you want in natural language!

In [ ]:
# Initialize agent
agent = ForecastAgent(verbose=True)

# Generate workflow from natural language
workflow = agent.generate_workflow(
    data=train,
    request="Forecast daily sales with weekly seasonality"
)

print("\n✅ Workflow generated!")

### What the Agent Analyzed

Let's see what the agent discovered about our data:

In [ ]:
# Access the analysis results
info = agent.last_workflow_info
data_chars = info['data_characteristics']

print("📊 DATA ANALYSIS:")
print(f"  Frequency: {data_chars['frequency']}")
print(f"  Seasonality Detected: {data_chars['seasonality']['detected']}")
print(f"  Seasonal Strength: {data_chars['seasonality']['strength']:.2%}")
print(f"  Seasonal Period: {data_chars['seasonality']['period']} days")
print(f"  Trend: {data_chars['trend']['direction']} (strength={data_chars['trend']['strength']:.2%})")
print(f"  Autocorrelation (lag-1): {data_chars['autocorrelation']['lag_1']:.2f}")
print(f"  Missing Data: {data_chars['missing_rate']:.1%}")
print(f"  Outliers: {data_chars['outlier_rate']:.1%}")

In [ ]:
# See the recommended model
print("\n🤖 RECOMMENDED MODEL:")
print(f"  Model: {info['model_type']}")
print(f"\n📝 GENERATED RECIPE:")
print(info['recipe_code'])

### Fit and Predict

Now use the generated workflow like any py-tidymodels workflow:

In [ ]:
# Fit the workflow
fit = workflow.fit(train)

# Make predictions on test data
predictions = fit.predict(test)

print("✅ Model fitted and predictions made!")
print(f"\nPredictions shape: {predictions.shape}")
print(predictions.head())

In [ ]:
# Visualize predictions
plt.figure(figsize=(12, 4))
plt.plot(train['date'].iloc[-60:], train['sales'].iloc[-60:], label='Train (last 60 days)', alpha=0.7)
plt.plot(test['date'], test['sales'], label='Actual', alpha=0.7)
plt.plot(test['date'], predictions['.pred'], label='Predicted', alpha=0.7, linestyle='--')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.title('Forecast Results')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### Performance Debugging

Let the agent analyze the model's performance:

In [ ]:
# Debug the model
diagnostics = agent.debug_session(fit, test)

print("\n💡 RECOMMENDATIONS:")
for i, rec in enumerate(diagnostics['recommendations'], 1):
    print(f"{i}. {rec}")

## Example 2: Grouped Forecasting (Multiple Stores)

Let's forecast sales for multiple stores simultaneously.

In [ ]:
# Create multi-store data
np.random.seed(42)
stores = ['Store_A', 'Store_B', 'Store_C']
dates = pd.date_range('2020-01-01', periods=365, freq='D')

data_list = []
for store in stores:
    # Each store has different baseline and trend
    baseline = np.random.uniform(80, 120)
    trend_strength = np.random.uniform(0.05, 0.15)
    
    trend = np.arange(len(dates)) * trend_strength
    seasonality = np.sin(np.arange(len(dates)) * 2 * np.pi / 7) * 15
    noise = np.random.randn(len(dates)) * 5
    sales = baseline + trend + seasonality + noise
    
    store_df = pd.DataFrame({
        'date': dates,
        'store_id': store,
        'sales': sales
    })
    data_list.append(store_df)

multi_store_df = pd.concat(data_list, ignore_index=True)

# Split train/test
train_multi = multi_store_df.groupby('store_id').apply(lambda x: x.iloc[:int(len(x)*0.8)]).reset_index(drop=True)
test_multi = multi_store_df.groupby('store_id').apply(lambda x: x.iloc[int(len(x)*0.8):]).reset_index(drop=True)

print(f"Total rows: {len(multi_store_df)}")
print(f"Stores: {multi_store_df['store_id'].unique()}")
print(f"\nSample:")
print(train_multi.head())

In [ ]:
# Generate workflow for grouped forecasting
agent_multi = ForecastAgent(verbose=True)

workflow_multi = agent_multi.generate_workflow(
    data=train_multi,
    request="Forecast sales for each store with weekly seasonality"
)

print("\n✅ Multi-store workflow generated!")

In [ ]:
# Fit with grouped modeling
fit_multi = workflow_multi.fit_nested(train_multi, group_col='store_id')

# Predict for all stores
predictions_multi = fit_multi.predict(test_multi)

print("✅ Fitted models for all stores!")
print(f"\nPredictions shape: {predictions_multi.shape}")

In [ ]:
# Visualize per-store forecasts
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, store in enumerate(stores):
    ax = axes[idx]
    
    train_store = train_multi[train_multi['store_id'] == store].iloc[-60:]
    test_store = test_multi[test_multi['store_id'] == store]
    pred_store = predictions_multi[predictions_multi['store_id'] == store]
    
    ax.plot(train_store['date'], train_store['sales'], label='Train', alpha=0.7)
    ax.plot(test_store['date'], test_store['sales'], label='Actual', alpha=0.7)
    ax.plot(pred_store['date'], pred_store['.pred'], label='Predicted', linestyle='--', alpha=0.7)
    
    ax.set_title(store)
    ax.set_xlabel('Date')
    ax.set_ylabel('Sales')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Example 3: Conversational Session

Build workflows through conversation:

In [ ]:
# Start conversational session
session = agent.start_session()

# Provide data to session
session.context['data'] = train

# Multi-turn conversation
session.send("I need to forecast sales")
session.send("Daily data")
session.send("Forecast 30 days ahead")

# Generate workflow from conversation
workflow_conv = session.get_workflow()

print("\n✅ Workflow generated from conversation!")

## Example 4: Custom Constraints

Specify constraints for model selection:

In [ ]:
# Generate workflow with constraints
agent_constrained = ForecastAgent(verbose=True)

workflow_constrained = agent_constrained.generate_workflow(
    data=train,
    request="Forecast sales",
    constraints={
        'max_train_time': 10,  # 10 seconds max
        'interpretability': 'high',  # Must be highly interpretable
        'max_memory': 100  # 100 MB max memory
    }
)

print("\n✅ Constrained workflow generated!")
print(f"Selected model: {agent_constrained.last_workflow_info['model_type']}")

## Summary

The `py_agent` package provides:

1. **Automatic Data Analysis**: Frequency, seasonality, trend detection
2. **Intelligent Model Selection**: Based on data characteristics and constraints
3. **Recipe Generation**: Automatic preprocessing based on data quality
4. **Workflow Creation**: Complete py-tidymodels workflows from natural language
5. **Performance Debugging**: Identify and fix common issues
6. **Conversational Interface**: Build workflows through dialogue

### Next Steps

- Try with your own data
- Experiment with different constraints
- Use debug_session() to improve models
- Explore conversational interface for complex scenarios

For more information, see: `py_agent/README.md`